In [0]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [code]
#Loading the Data again
data= pd.read_csv('../input/ml-challenge-turkiye-is-bankasi-2/train.csv')
test = pd.read_csv('../input/ml-challenge-turkiye-is-bankasi-2/test_set.csv')
guvenendexi=pd.read_excel('../input/guvenednx/ekonomikgvenendexi.xls')
#tüfe=pd.read_excel('../input/tufedegerleri/TFE.xls')

# %% [code]
# Import label encoder 
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
data['ISLEM_TURU']= label_encoder.fit_transform(data.ISLEM_TURU) 

# %% [code]
data['YIL_AY'] = pd.to_datetime(data['YIL_AY'], format='%Y%m')
test['YIL_AY'] = pd.to_datetime(test['YIL_AY'], format='%Y%m')
#tüfe['YIL_AY'] = pd.to_datetime(tüfe['YIL_AY'], format='%m-%Y')

# %% [code]
data['yıl'] = data['YIL_AY'].dt.year
test['yıl'] = test['YIL_AY'].dt.year
data['ay'] = data['YIL_AY'].dt.month
test['ay'] = test['YIL_AY'].dt.month
#tüfe['yıl'] = tüfe['YIL_AY'].dt.year
#tüfe['ay'] = tüfe['YIL_AY'].dt.month

# %% [code]
#tüfe.drop('YIL_AY',axis=1,inplace=True)

# %% [code]
gvenendexi=guvenendexi.drop(['Tüketici güven endeksi','Ekonomik \ngüven\nendeksi','Genel\ngidişat'],axis=1,inplace=False)

# %% [code]
gvenendexi.columns =['yıl', 'ay','maddidurum']

# %% [code]
train=data.merge(gvenendexi)

# %% [code]
train= train.drop_duplicates()


# %% [code]
#train=data.merge(tüfe)

# %% [code]
train.head()

# %% [code]
from sklearn.model_selection import train_test_split
X=train.drop(['ISLEM_TUTARI','ISLEM_ADEDI','YIL_AY','SEKTOR','Record_Count','yıl','ay'], axis='columns' ,inplace=False)
y=train.ISLEM_TUTARI
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)


# %% [code]
from xgboost import XGBRegressor

my_model = XGBRegressor(learning_rate=0.5,max_depth=10)
my_model.fit(X_train, y_train)

# %% [code]
from sklearn.metrics import mean_absolute_error
predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))
from sklearn.metrics import mean_squared_error
from math import sqrt

print("Root Mean Squared Error: " + str(np.sqrt(mean_squared_error(predictions, y_valid))))

# %% [code]

from sklearn import preprocessing. 
label_encoder = preprocessing.LabelEncoder()
test['ISLEM_TURU']= label_encoder.fit_transform(test.ISLEM_TURU) 

# %% [code]
test=test.merge(gvenendexi)

# %% [code]
test= test.drop_duplicates()

# %% [code]
#test=test.merge(tüfe)

# %% [code]
X_test_prep=test.drop(['ISLEM_TUTARI','ISLEM_ADEDI','YIL_AY','SEKTOR','Record_Count','yıl','ay'], axis='columns' ,inplace=False)

# %% [code]
test_preds = my_model.predict(X_test_prep)

# %% [code]
output = pd.DataFrame({'Sektor_Id': test.Id, 'Expected': test_preds}) 
df = output.groupby(['Sektor_Id'])["Expected"].sum().reset_index()
df.to_csv('submission.csv', index=False)